# Montamos Unidad de Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Colab Notebooks/TP_Ind4

/content/drive/MyDrive/Colab Notebooks/TP_Ind4


In [ ]:
%cd drive/MyDrive/TP_Ind4

/content/drive/.shortcut-targets-by-id/17FeAVfVLm-bhQ0cBKXS7bsPk0E6kXi_t/TP_Ind4


In [3]:
%ls

 AnalisisExploratorio.ipynb
 data/
 Docs.gdoc
'model tunning'/
'TP 1 - Exploración, visualización de datos y Machine Learning.pdf'


# Paquetes

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from plotnine import *

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [22]:
# Importamos el set de entrenamiento.
train = pd.read_csv("data/Train.csv", index_col=0)

In [23]:
train.shape

(33908, 109)

In [24]:
# Separamos entre variables explicativas y respuesta.
X_train = train.drop('Subscription', axis = 1)
y_train = train['Subscription']

## A tener en cuenta
Cuando tuneamos un modelo:
* **Siempre** incluir todos sus **hiperparámetros en el grid**, aunque sea con el valor default. Esto hace que el dataframe que generamos con los resultados tenga toda la información y, si después queremos tunear un hiperparámetro que no habíamos tuneado, **no se agrega una nueva columna**.
* **Siempre** incluir el *train score* para ver si tenemos overfitting, comparando con el resultado de test.

In [25]:
# Grid de valores de hiperparámetros.
knn_grid = {'n_neighbors':[1,3],
            'weights':["uniform", "distance"]}

In [26]:
# Hiperparámetros fijos del modelo entrenado.
knn = KNeighborsClassifier(n_jobs=-1)

In [ ]:
time_start = time.time()

# Grid Search
knn_cv = GridSearchCV(knn, knn_grid, cv = 5, scoring='roc_auc', verbose=4, n_jobs=-1, return_train_score=True, refit=True)

# Randomized Search
#svm_cv = RandomizedSearchCV(svm, svm_grid, n_iter = 500, cv = 10, scoring='accuracy', verbose=10, n_jobs=-1, random_state=354)

knn_cv.fit(X_train, y_train.values.ravel())

# Generamos y mostramos un Dataframe con los resultados del GridSearch
knn_cv_results = pd.DataFrame(knn_cv.cv_results_)
print(knn_cv_results.head(10))

# Generamos un csv con los resultados para el modelo.
knn_cv_results.to_csv('model tunning/KNN/KNN_CVResults.csv')
#tree_cv_results.to_csv('ClassificationTree/Tree_CVResults.csv', mode='a', header=False)

# Imprimimos el tiempo total de duración de Cross-Validation.
time_finish = time.time()
print(f'Duración {round(time_finish - time_start, 5)} segundos')
print(f'Duración {round((time_finish - time_start)/60, 5)} minutos')

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [27]:
train

,Age,Subscription,Balance_pos_log,Last Contact Duration Log,Campaign Log,Job_admin.,Job_blue-collar,Job_management,Job_others,Job_retired,Job_services,Job_technician,Marital Status_divorced,Marital Status_married,Marital Status_single,Education_18,Education_19,Education_20,Education_21,Education_22,Education_23,Education_24,Education_25,Education_26,Education_27,Education_28,Education_29,Education_30,Education_31,Education_32,Education_33,Education_34,Education_35,Education_36,Education_37,Education_38,Education_39,Education_40,Education_41,Education_42,...,Education_72,Education_73,Education_74,Education_75,Education_76,Education_77,Education_78,Education_79,Education_80,Education_81,Education_82,Education_83,Education_84,Education_85,Education_86,Education_87,Education_88,Education_89,Education_90,Education_92,Education_93,Education_94,Education_95,Last Contact Month_apr,Last Contact Month_aug,Last Contact Month_feb,Last Contact Month_jan,Last Contact Month_jul,Last Contact Month_jun,Last Contact Month_may,Last Contact Month_nov,Last Contact Month_other,"Last Contact Day Cat_(0, 7]","Last Contact Day Cat_(7, 14]","Last Contact Day Cat_(14, 21]","Last Contact Day Cat_(21, 31]",Housing Loan_yes,Personal Loan_yes,Age_60_True,Balance_neg_cat_True
24915,46,0,2.514548,2.161398,0.603144,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
11794,33,0,2.494155,2.179006,0.303196,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0
1279,35,0,-2.000000,2.725103,0.478566,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1
913,47,0,2.243038,2.008643,0.845718,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
5554,32,1,3.238548,3.272076,0.478566,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,53,1,1.724276,2.587722,0.004321,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
26216,39,1,2.693727,2.334474,0.004321,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
15941,57,0,2.531479,2.222742,0.303196,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
16952,35,1,3.516535,3.191733,0.303196,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
'''from google.colab import output
output.eval_js('new Audio("https://upload.wikipedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')'''

In [ ]:
tree_cv_results.sort_values('rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ccp_alpha,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
289,0.842763,0.013487,0.014977,0.000241,0,gini,None,None,None,100,best,"{'ccp_alpha': 0, 'criterion': 'gini', 'max_dep...",0.914496,0.911177,0.914296,0.908036,0.912514,0.912104,0.002371,1,0.940052,0.940925,0.940018,0.940722,0.939910,0.940325,0.000414
257,0.888702,0.011909,0.015290,0.000192,0,gini,30,None,None,100,best,"{'ccp_alpha': 0, 'criterion': 'gini', 'max_dep...",0.914496,0.911177,0.914296,0.908036,0.912461,0.912093,0.002369,2,0.940052,0.940925,0.940018,0.940722,0.939895,0.940322,0.000417
130,0.669635,0.012075,0.014002,0.000263,0,gini,13,None,None,200,best,"{'ccp_alpha': 0, 'criterion': 'gini', 'max_dep...",0.913339,0.910609,0.913760,0.910151,0.912463,0.912064,0.001445,3,0.927471,0.928709,0.927480,0.928088,0.928279,0.928005,0.000477
145,0.468077,0.016854,0.014507,0.000260,0,gini,13,0.7,None,100,best,"{'ccp_alpha': 0, 'criterion': 'gini', 'max_dep...",0.914087,0.910592,0.913713,0.910448,0.911365,0.912041,0.001554,4,0.930093,0.931397,0.931427,0.931675,0.931554,0.931229,0.000577
225,0.845211,0.013868,0.015047,0.000183,0,gini,25,None,None,100,best,"{'ccp_alpha': 0, 'criterion': 'gini', 'max_dep...",0.914310,0.911132,0.913784,0.908048,0.912475,0.911950,0.002241,5,0.939993,0.940801,0.939756,0.940663,0.939433,0.940129,0.000525


In [ ]:
hyperparams = ['param_ccp_alpha', 'param_criterion', 'param_max_depth', 'param_max_features', 'param_max_leaf_nodes', 'param_min_samples_leaf', 'param_splitter']
for hyperparam in hyperparams:
  print(tree_cv_results.groupby([hyperparam]).agg({'mean_test_score':'mean'}))

                 mean_test_score
param_ccp_alpha                 
0.000                   0.900994
0.001                   0.902618
0.010                   0.883227
0.100                   0.500000
                 mean_test_score
param_criterion                 
entropy                 0.800848
gini                    0.792571
                 mean_test_score
param_max_depth                 
5                       0.798218
7                       0.797048
9                       0.797221
11                      0.796956
13                      0.796897
15                      0.796909
20                      0.796274
25                      0.796003
30                      0.795878
                    mean_test_score
param_max_features                 
0.7                          0.7933
                      mean_test_score
param_max_leaf_nodes                 
10                           0.785959
20                           0.799182
40                           0.800812
         

# Results

In [ ]:
# Importamos los resultados.
results = pd.read_csv('ClassificationTree/Tree_CVResults.csv', index_col = 0)

In [ ]:
results.shape

(2560, 27)

In [ ]:
# Seleccionamos solamente las columnas de los hiperparámetros y resultados de train y CV.
features = ['param_ccp_alpha', 
            'param_criterion', 
            'param_max_depth', 
            'param_max_features', 
            'param_max_leaf_nodes', 
            'param_min_samples_leaf', 
            'param_splitter',
            'mean_test_score', 
            'mean_train_score']
# Reseteamos el índice del df.
results = results[features].reset_index(drop = True)
# Completamos los NAs con 'None'.
results.fillna(value = 'None', inplace = True)

In [ ]:
results.sort_values('mean_test_score', ascending = False).drop_duplicates().head(25)

,param_ccp_alpha,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,param_splitter,mean_test_score,mean_train_score
289,0.0,gini,None,None,None,100,best,0.912104,0.940325
257,0.0,gini,30,None,None,100,best,0.912093,0.940322
130,0.0,gini,13,None,None,200,best,0.912064,0.928005
145,0.0,gini,13,0.7,None,100,best,0.912041,0.931229
225,0.0,gini,25,None,None,100,best,0.911950,0.940129
609,0.0,entropy,None,None,None,100,best,0.911938,0.940690
577,0.0,entropy,30,None,None,100,best,0.911925,0.940654
626,0.0,entropy,None,0.7,None,200,best,0.911894,0.929361
65,0.0,gini,9,None,None,100,best,0.911879,0.924540
465,0.0,entropy,13,0.7,None,100,best,0.911847,0.931360


In [ ]:
hyperparams = ['param_ccp_alpha', 'param_criterion', 'param_max_depth', 'param_max_features', 'param_max_leaf_nodes', 'param_min_samples_leaf', 'param_splitter']
for hyperparam in hyperparams:
  print(results.groupby([hyperparam]).agg({'mean_test_score':'mean'}))

                 mean_test_score
param_ccp_alpha                 
0.000                   0.900994
0.001                   0.902618
0.010                   0.883227
0.100                   0.500000
                 mean_test_score
param_criterion                 
entropy                 0.800848
gini                    0.792571
                 mean_test_score
param_max_depth                 
5.0                     0.798218
7.0                     0.797048
9.0                     0.797221
11.0                    0.796956
13.0                    0.796897
15.0                    0.796909
20.0                    0.796274
25.0                    0.796003
30.0                    0.795878
None                    0.795691
                    mean_test_score
param_max_features                 
0.7                        0.793300
None                       0.800119
                      mean_test_score
param_max_leaf_nodes                 
10.0                         0.785959
20.0            